# Setup + Login (no quit)

In [ ]:
# 📦 Install necessary packages (if not already installed)
!pip install selenium webdriver-manager

# 📚 Import libraries
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
import time

# 🔧 Configuration (🔑 enter your credentials here) ⚠️ Warning: The sample account below has expired 🚫

EMAIL = "aj.s.h.d.gsks.g.sh.d.g.ff@gmail.com"
PASSWORD = "NZcFuR_g7h9!~w6"
KEYWORD_FILE = "data.txt"  # make sure this file exists in your working directory

# 🖥️ Setup browser
options = Options()
options.add_argument('--start-maximized')
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=options)
wait = WebDriverWait(driver, 25)

# 🛂 Login function
def login(driver, wait, email, password):
    driver.get("https://app.pinsearch.co/sign-in")
    wait.until(EC.presence_of_element_located((By.ID, "identifier-field"))).send_keys(email)
    wait.until(EC.presence_of_element_located((By.ID, "password-field"))).send_keys(password)

    continue_btn = driver.execute_script("""
        return document.querySelector("body > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > form:nth-child(3) > button:nth-child(3)");
    """)
    wait.until(lambda d: continue_btn.is_enabled())
    continue_btn.click()

    try:
        popup = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[5]/button/*[name()='svg']")))
        popup.click()
        print("✅ Closed popup.")
    except:
        print("⚠️ No popup appeared.")

    wait.until(EC.presence_of_element_located((By.XPATH, "//a[normalize-space()='Keyword Finder']")))
    print("✅ Logged in successfully.")

# 🚪 Perform login (browser will remain open)
login(driver, wait, EMAIL, PASSWORD)


# Keyword Scraper (no login again)

In [ ]:
# This cell ONLY depends on already running driver and wait
# DO NOT re-import selenium or re-initialize driver

def run_scraper(driver, wait, keyword_file):
    with open(keyword_file, "r", encoding="utf-8") as f:
        keywords = [kw.strip() for kw in f if kw.strip()]

    print(f"🚀 Starting scraper - {len(keywords)} keywords found.")

    for idx, keyword in enumerate(keywords, 1):
        print(f"\n👉 [{idx}/{len(keywords)}] Processing keyword: {keyword}")

        driver.get("https://app.pinsearch.co/keyword-research")
        print("📄 Opened Keyword Finder page.")

        try:
            search_input = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//input[@placeholder='Enter your seed keyword']")))
            search_input.clear()
            search_input.send_keys(keyword)
            print(f"✍️ Entered keyword: {keyword}")

            search_button = wait.until(EC.element_to_be_clickable(
                (By.XPATH, "//button[normalize-space()='Search']")))
            search_button.click()
            print(f"🔍 Clicked Search button.")

        except Exception as e:
            print(f"❌ Error entering keyword: {e}")
            continue

        # 🛑 Check error popup
        time.sleep(3)
        try:
            popup = driver.find_element(By.XPATH, "//li[@data-type='error']")
            if popup.is_displayed():
                popup_title = popup.find_element(By.XPATH, ".//div[@data-title]").text
                if "Couldn't find keyword suggestions" in popup_title:
                    print(f"⚠️ No keyword suggestions found for '{keyword}'. Skipping...")
                    continue
        except:
            pass  # No popup — good

        # ✅ Wait for Select All
        try:
            WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Select all']"))
            )
            print("✅ Search results loaded.")
        except Exception as e:
            print(f"❌ No search results for '{keyword}'. Skipping...")
            continue

        # 🖱️ Export
        try:
            select_all = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Select all']")))
            select_all.click()
            print("🖱️ Clicked Select All.")

            export_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Export')]")))
            export_button.click()
            print("📥 Clicked Export button.")

            all_keywords_button = wait.until(EC.element_to_be_clickable(
                (By.XPATH, "//button[contains(text(),'All Keywords')]")))
            all_keywords_button.click()
            print("📄 Download started.")

        except Exception as e:
            print(f"❌ Export error for '{keyword}': {e}")
            continue

        print(f"✅ Finished keyword: {keyword}")
        time.sleep(2)  # Allow download time

    print("\n🏁 Finished scraping all keywords.")

# 🏃 Run scraper
run_scraper(driver, wait, KEYWORD_FILE)
